In [ ]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

In [8]:
from google.colab import files
uploaded = files.upload()

Saving players_data-2024_2025.csv to players_data-2024_2025 (1).csv


In [152]:
df = pd.read_csv('players_data-2024_2025.csv')

if 'Born' in df.columns:
    df['Age'] = 2025 - df['Born']

df = df[df['90s'] >= 15].dropna(subset=['Pos'])

if 'GA' in df.columns:
    df['GA_per90'] = df['GA'] / df['90s']

base_cols = ['Player', 'Nation', 'Age', 'Pos', 'Squad', 'Comp', '90s']
attacking = ['Gls', 'Ast', 'G+A', 'xG', 'xAG', 'npxG', 'G-PK']
defending = ['Tkl', 'TklW', 'Blocks', 'Int', 'Tkl+Int', 'Clr', 'Err']
passing = ['PrgP', 'PrgC', 'KP', 'Cmp%_stats_passing', 'Ast_stats_passing', 'xA', 'PPA']
goalkeeping = ['GA', 'Saves', 'Save%', 'CS', 'CS%', 'PKA', 'PKsv']
possession = ['Touches', 'Carries', 'PrgR', 'Mis', 'Dis']
misc = ['CrdY', 'CrdR', 'PKwon', 'PKcon', 'Recov']

cols = base_cols + attacking + defending + passing + goalkeeping + possession + misc

df = df[[col for col in cols if col in df.columns]]

df.dropna(subset=['90s', 'Pos'], inplace=True)

conn = sqlite3.connect(':memory:')
df.to_sql('players', conn, index=False, if_exists='replace')

1075

In [153]:
query_gk = """
SELECT
  Player, Squad, Age, "90s",
  ROUND("Save%" , 2) AS Save_pct,
  ROUND(Saves / "90s", 2) AS Saves_per90,
  ROUND(CS / "90s", 2) AS CS_per90,
  ROUND(PKsv / "90s", 2) AS PKsv_per90,
  ROUND(GA / "90s", 2) AS GA_per90
FROM players
WHERE Pos = 'GK' AND "90s" >= 25
"""

gk_df = pd.read_sql(query_gk, conn)

# Metrics to include
gk_cols = ['Save_pct', 'Saves_per90', 'CS_per90', 'PKsv_per90']
for col in gk_cols:
    gk_df[col + '_rank'] = gk_df[col].rank(ascending=False)

# GA is bad → ascending rank
gk_df['GA_per90_rank'] = gk_df['GA_per90'].rank(ascending=True)

# Unweighted average rank
gk_df['avg_rank'] = gk_df[[col + '_rank' for col in gk_cols] + ['GA_per90_rank']].mean(axis=1)

top_gk = gk_df.sort_values('avg_rank').head(1)
top_gk[['Player', 'Squad', 'Age', '90s', 'Save_pct', 'GA_per90', 'CS_per90', 'PKsv_per90', 'avg_rank']]


,Player,Squad,Age,90s,Save_pct,GA_per90,CS_per90,PKsv_per90,avg_rank
45,Đorđe Petrović,Strasbourg,26.0,28.0,80.1,1.14,0.36,0.04,11.9


In [154]:
query_defenders = """
SELECT
  Player, Squad, Age, "90s",
  ROUND("Tkl+Int" / "90s", 2) AS TklInt_per90,
  ROUND(TklW / "90s", 2) AS TklW_per90,
  ROUND(Blocks / "90s", 2) AS Blocks_per90,
  ROUND(Clr / "90s", 2) AS Clr_per90,
  ROUND(Recov / "90s", 2) AS Recov_per90,
  ROUND(PrgP / "90s", 2) AS PrgP_per90,
  ROUND(PrgC / "90s", 2) AS PrgC_per90,
  ROUND(KP / "90s", 2) AS KP_per90,
  ROUND(PPA / "90s", 2) AS PPA_per90,
  ROUND(GA / "90s", 2) AS GA_per90
FROM players
WHERE Pos = 'DF' AND "90s" >= 24
"""

df_df = pd.read_sql(query_defenders, conn)

df_cols = ['TklInt_per90', 'TklW_per90', 'Blocks_per90', 'Clr_per90',
           'Recov_per90', 'PrgP_per90', 'PrgC_per90', 'KP_per90', 'PPA_per90']

for col in df_cols:
    df_df[col + '_rank'] = df_df[col].rank(ascending=False)

df_df['GA_per90_rank'] = df_df['GA_per90'].rank(ascending=True)
df_df['avg_rank'] = df_df[[c + '_rank' for c in df_cols] + ['GA_per90_rank']].mean(axis=1)

top_3_defenders = df_df.sort_values('avg_rank').head(3)
top_3_defenders[['Player', 'Squad', 'Age', '90s'] + df_cols + ['GA_per90']]


,Player,Squad,Age,90s,TklInt_per90,TklW_per90,Blocks_per90,Clr_per90,Recov_per90,PrgP_per90,PrgC_per90,KP_per90,PPA_per90,GA_per90
122,Maximilian Mittelstädt,Stuttgart,28.0,24.7,4.70,2.15,1.13,3.56,4.33,6.60,2.15,1.66,2.23,None
136,Pedro Porro,Tottenham,26.0,27.4,3.54,1.68,2.59,2.92,5.40,5.36,2.30,1.97,2.01,None
6,Trent Alexander-Arnold,Liverpool,27.0,24.9,4.02,1.97,1.41,2.17,5.06,8.47,1.97,2.01,2.49,None


In [155]:
query_mf_stats = """
SELECT
  Player, Squad, Age, "90s",
  ROUND(Gls / "90s", 2) AS Gls_per90,
  ROUND(Ast / "90s", 2) AS Ast_per90,
  ROUND(xA / "90s", 2) AS xA_per90,
  ROUND(xAG / "90s", 2) AS xAG_per90,
  ROUND(KP / "90s", 2) AS KP_per90,
  ROUND(PrgP / "90s", 2) AS PrgP_per90,
  ROUND(PrgC / "90s", 2) AS PrgC_per90,
  ROUND(PPA / "90s", 2) AS PPA_per90,
  ROUND(Recov / "90s", 2) AS Recov_per90,
  ROUND(TklW / "90s", 2) AS TklW_per90,
  ROUND(Int / "90s", 2) AS Int_per90
FROM players
WHERE (Pos = 'MF' OR Pos = 'MF,FW') AND "90s" >= 25
"""

mf_df = pd.read_sql(query_mf_stats, conn)

stat_cols = [
    'Gls_per90', 'Ast_per90', 'xA_per90', 'xAG_per90',
    'KP_per90', 'PrgP_per90', 'PrgC_per90',
    'PPA_per90', 'Recov_per90', 'TklW_per90', 'Int_per90'
]

for col in stat_cols:
    mf_df[col + '_rank'] = mf_df[col].rank(ascending=False, method='min')
mf_df['avg_rank'] = mf_df[[col + '_rank' for col in stat_cols]].mean(axis=1)

top_4_mids = mf_df.sort_values('avg_rank').head(4)
top_4_mids[['Player', 'Squad', 'Age', '90s'] + stat_cols]


,Player,Squad,Age,90s,Gls_per90,Ast_per90,xA_per90,xAG_per90,KP_per90,PrgP_per90,PrgC_per90,PPA_per90,Recov_per90,TklW_per90,Int_per90
30,Bruno Fernandes,Manchester Utd,31.0,30.6,0.26,0.29,0.23,0.25,2.75,9.18,2.12,2.39,6.27,1.54,0.82
70,Pedri,Barcelona,23.0,28.1,0.14,0.14,0.30,0.24,2.06,10.50,2.74,2.42,8.08,1.07,0.89
24,Mikkel Damsgaard,Brentford,25.0,28.6,0.07,0.35,0.22,0.27,1.99,7.31,2.20,2.31,5.45,1.43,0.87
7,Alex Baena,Villarreal,24.0,25.0,0.24,0.28,0.45,0.39,3.36,6.72,3.16,2.64,5.20,0.56,0.84


NameError: name 'plot_percentile_comparison' is not defined

In [156]:
query_fw_stats = """
SELECT
  Player, Squad, Age, "90s",
  ROUND(Gls / "90s", 2) AS Gls_per90,
  ROUND(Ast / "90s", 2) AS Ast_per90,
  ROUND((Gls + Ast) / "90s", 2) AS GA_per90,
  ROUND(xG / "90s", 2) AS xG_per90,
  ROUND(npxG / "90s", 2) AS npxG_per90,
  ROUND((Gls - xG) / "90s", 2) AS xG_diff_per90,
  ROUND(xAG / "90s", 2) AS xAG_per90,
  ROUND(KP / "90s", 2) AS KP_per90,
  ROUND(PrgC / "90s", 2) AS PrgC_per90
FROM players
WHERE (Pos = 'FW' OR Pos = 'FW,MF') AND "90s" >= 25
"""

fw_df = pd.read_sql(query_fw_stats, conn)

# STRIKER RANKING
striker_cols = ['Gls_per90', 'xG_per90', 'GA_per90', 'xG_diff_per90', 'npxG_per90']
for col in striker_cols:
    fw_df[col + '_striker_rank'] = fw_df[col].rank(ascending=False)
fw_df['striker_avg_rank'] = fw_df[[col + '_striker_rank' for col in striker_cols]].mean(axis=1)
top_strikers = fw_df.sort_values('striker_avg_rank').head(2)
striker_players = top_strikers['Player'].tolist()

# WINGER RANKING (exclude strikers)
winger_cols = ['Ast_per90', 'xAG_per90', 'KP_per90', 'PrgC_per90', 'Gls_per90']
for col in winger_cols:
    fw_df[col + '_winger_rank'] = fw_df[col].rank(ascending=False)
fw_df['winger_avg_rank'] = fw_df[[col + '_winger_rank' for col in winger_cols]].mean(axis=1)

wingers_df = fw_df[~fw_df['Player'].isin(striker_players)]
top_winger = wingers_df.sort_values('winger_avg_rank').head(1)

# COMBINE FINAL FRONT 3
front_3 = pd.concat([top_strikers, top_winger])
front_3[['Player', 'Squad', 'Age', '90s', 'Gls_per90', 'Ast_per90', 'xG_per90', 'xG_diff_per90', 'KP_per90', 'PrgC_per90']]

,Player,Squad,Age,90s,Gls_per90,Ast_per90,xG_per90,xG_diff_per90,KP_per90,PrgC_per90
24,Harry Kane,Bayern Munich,32.0,25.1,0.96,0.32,0.78,0.18,1.31,1.12
46,Mohamed Salah,Liverpool,33.0,33.5,0.84,0.54,0.71,0.12,2.39,4.24
44,Raphinha,Barcelona,29.0,27.3,0.55,0.33,0.60,-0.05,3.04,3.08
